In [3]:
# selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import pymongo

In [3]:
start = 2
for i in range(start, 10):
    start = i
    print(i)

2
3
4
5
6
7
8
9


In [4]:
def getChromeDriver(options):
    opts = webdriver.ChromeOptions()
    for opt in options:
        opts.add_argument(opt)
    driver = webdriver.Chrome(options = opts)
    return driver

def isElementPresent(driver, by, value):
        """
        用来判断元素标签是否存在，
        """
        try:
            element = driver.find_element(by=by, value=value)
        except NoSuchElementException as e:
            # 发生了NoSuchElementException异常，说明页面中未找到该元素，返回False
            return False
        else:
            # 没有发生异常，表示在页面中找到了该元素，返回True
            return True

In [7]:
xsqkurl = r'https://kns.cnki.net/kcms/detail/detail.aspx?dbcode=CAPJ&dbname=CAPJLAST&filename=SKYK20220215000&uniplatform=NZKPT&v=ikHDFfO2iKKCkFV7XFP-sKn8U-o2LnmKQ7Docz92wH_z00FHeWgzP1HlcbehGqzW'
hyurl = r'https://kns.cnki.net/kcms/detail/detail.aspx?dbcode=CPFD&dbname=CPFDLAST2022&filename=GCKZ202107001229&uniplatform=NZKPT&v=EWFnAIc5r6Tl8jZzgiXAa5wG8c3-RKLr3d5PGoZ4xw3DAv3lnDVhq1A2-kxTeucS1nlHHPnD3kw%3d'

In [8]:
url = xsqkurl

In [ ]:
url = hyurl

In [11]:
options = [
    '--disable-gpu',
    '--blink-settings=imagesEnabled=false',
    # '--headless',
    ]
driver = getChromeDriver(options)

driver.get(url)
homePage = driver.current_window_handle
driver.switch_to.window(homePage)
for i in range(4):
    driver.execute_script('window.open()')
driver.switch_to.window(homePage)

In [13]:
windowsLen = len(driver.window_handles)
while len(driver.window_handles) != 1:
    driver.switch_to.window(driver.window_handles[1])
    driver.close()
    driver.switch_to.window(homePage)

In [10]:
driver.quit()

In [ ]:
# 学术期刊
xsqkInfoCSS = {
    'qk_name': r'div.top-tip > span > a',        # 所属期刊名称及时间(可能没有时间)
    'qk_netTime': r'div.head-time > span',       # 文献网络首发时间
    'qk_title': r'div.wx-tit > h1',              # 文献标题
    'qk_auther': r'h3#authorpart.author > span', # 作者
    'qk_institution': r'a.author',               # 作者所属机构
    'qk_abstract': r'span#ChDivSummary',         # 摘要
    'qk_keys': r'p.keywords > a'                 # 关键字
    }

# 会议
hyInfoCSS = {
    'hy_title': r'div.wx-tit > h1',              # 文献标题
    'hy_auther': r'h3#authorpart.author > span', # 作者
    'hy_institution': r'a.author',               # 作者所属机构
    'hy_abstract': r'span#ChDivSummary',         # 摘要
    'hy_keys': r'p.keywords > a'                 # 关键字
}
hy_info = r'div.row > span'                      # 会议信息（名称、时间、地点等）


In [ ]:
# 学术期刊
moreEle = driver.find_element(by = By.CSS_SELECTOR, value = 'a#ChDivSummaryMore')
if(moreEle.get_attribute('style') != 'display: none;'):
    moreEle.click()
for css in xsqkInfoCSS:
    if isElementPresent(driver, By.CSS_SELECTOR, xsqkInfoCSS[css]):
        eles = driver.find_elements(by = By.CSS_SELECTOR, value = xsqkInfoCSS[css])
        print('{}:'.format(css))
        for ele in eles:
            print(ele.text, end = ' ')
        print(end = '\n')

In [ ]:
# 会议
save = {}
moreEle = driver.find_element(by = By.CSS_SELECTOR, value = 'a#ChDivSummaryMore')
if(moreEle.get_attribute('style') != 'display: none;'):
    moreEle.click()
for css in hyInfoCSS:
    if isElementPresent(driver, By.CSS_SELECTOR, hyInfoCSS[css]):
        eles = driver.find_elements(by = By.CSS_SELECTOR, value = hyInfoCSS[css])
        infoStr = ''
        for ele in eles:
            infoStr += ele.text
            infoStr += '  '
        save[css] = infoStr
if isElementPresent(driver, By.CSS_SELECTOR, hy_info):
    eles = driver.find_elements(by = By.CSS_SELECTOR, value = hy_info)
    for ele in eles:
        if(ele.text == '会议名称：'):
            infoStr = ele.find_element(by = By.XPATH, value = '../p').text
            save['hy_hyname'] = infoStr
            num = 0
            total = len(infoStr)
            for char in infoStr:
                if char >= '\u4e00' and char <= '\u9fa5':
                    print(char)
                    num += 1
            print(infoStr)
            print(num/total)
        elif(ele.text == '会议时间：'):
            infoStr = ele.find_element(by = By.XPATH, value = '../p').text
            save['hy_hytime'] = infoStr
        elif(ele.text == '会议地点：'):
            infoStr = ele.find_element(by = By.XPATH, value = '../p').text
            save['hy_hyplace'] = infoStr

client = pymongo.MongoClient('localhost', 27017)    # 连接服务器，需要先开启服务
db = client['mytest']  # 选择数据库
db.myi.insert_one(save)
data_all = db.myi.find()
for d in data_all:
    print(d)